# qBraid-SDK: Transpiler

In [1]:
import numpy as np
from time import time

from qbraid import __version__ as qbraid_version
from qbraid import circuit_wrapper, SUPPORTED_PROGRAM_TYPES
from qbraid.interface import to_unitary, circuits_allclose, draw
from qbraid.interface.programs import bell_data

start_notebook = time()

In [2]:
qbraid_version

'0.1.4'

The qbraid transpiler supports the following frontend modules / quantum program types:

In [3]:
SUPPORTED_PROGRAM_TYPES

{'cirq': 'Circuit',
 'pyquil': 'Program',
 'qiskit': 'QuantumCircuit',
 'braket': 'Circuit',
 'pennylane': 'QuantumTape'}

## Intro to `qbraid.circuit_wrapper` with Bell circuit example

Load test data containing a bell circuit function for each supported program type, along with the unitary matrix that each circuit represents / implements.

In [4]:
bell_circuits, expected_u = bell_data()
print(expected_u.shape)

(4, 4)


Randomly choose a source and target package

In [5]:
pkgs = list(SUPPORTED_PROGRAM_TYPES.keys())
source = pkgs.pop(np.random.randint(len(pkgs)))
target = pkgs.pop(np.random.randint(len(pkgs) - 1))
print(f"{source} --> {target}")

pennylane --> qiskit


Instantiate the circuit object in the source package and print its diagram

In [6]:
source_bell = bell_circuits[source]()
print(f"{type(source_bell)}\n")
draw(source_bell)

<class 'pennylane.tape.tape.QuantumTape'>

0: ──H─╭●─┤  
1: ────╰X─┤  


Apply the qbraid circuit wrapper to the source circuit

In [7]:
wrapped_circuit = circuit_wrapper(source_bell)
print(type(wrapped_circuit))

<class 'qbraid.transpiler.wrappers.pennylane_qtape.PennylaneQTapeWrapper'>


Use the transpile method to convert to the target circuit type and prints its diagram

In [8]:
target_bell = wrapped_circuit.transpile(target)
print(f"{type(target_bell)}\n")
draw(target_bell)

<class 'qiskit.circuit.quantumcircuit.QuantumCircuit'>

          ┌───┐
q_0: ─────┤ X ├
     ┌───┐└─┬─┘
q_1: ┤ H ├──■──
     └───┘     


Next we'll verify that the unitary representation of the target circuit is correct.

Here, we're using the sdk's `to_unitary` function, which interprets the type of the input circuit, calculates its unitary using methods from that circuits native module, and returns the resulting `numpy.ndarray`.

In [9]:
target_u = to_unitary(target_bell)
np.allclose(target_u, expected_u)

True

## Now, a non-trivial example

In [10]:
from qiskit import QuantumCircuit

In [11]:
def test_circuit():
    circuit = QuantumCircuit(4)

    circuit.h([0, 1, 2, 3])
    circuit.x([0, 1])
    circuit.y(2)
    circuit.z(3)
    circuit.s(0)
    circuit.sdg(1)
    circuit.t(2)
    circuit.tdg(3)
    circuit.rx(np.pi / 4, 0)
    circuit.ry(np.pi / 2, 1)
    circuit.rz(3 * np.pi / 4, 2)
    circuit.p(np.pi / 8, 3)
    circuit.sx(0)
    circuit.sxdg(1)
    circuit.iswap(2, 3)
    circuit.swap([0, 1], [2, 3])
    circuit.cx(0, 1)
    circuit.cp(np.pi / 4, 2, 3)

    return circuit

We'll start with a 4-qubit qiskit circuit that uses 15 unique gates

In [12]:
qiskit_circuit = test_circuit()
print(f"{type(qiskit_circuit)}\n")
qiskit_circuit.draw()

<class 'qiskit.circuit.quantumcircuit.QuantumCircuit'>



┌───┐┌───┐ ┌───┐ ┌─────────┐   ┌────┐                 
q_0: ┤ H ├┤ X ├─┤ S ├─┤ Rx(π/4) ├───┤ √X ├───X────────■────
     ├───┤├───┤┌┴───┴┐├─────────┤  ┌┴────┴┐  │      ┌─┴─┐  
q_1: ┤ H ├┤ X ├┤ Sdg ├┤ Ry(π/2) ├──┤ √Xdg ├──┼──X───┤ X ├──
     ├───┤├───┤└┬───┬┘├─────────┴┐┌┴──────┴┐ │  │   └───┘  
q_2: ┤ H ├┤ Y ├─┤ T ├─┤ Rz(3π/4) ├┤0       ├─X──┼──■───────
     ├───┤├───┤┌┴───┴┐└┬────────┬┘│  Iswap │    │  │P(π/4) 
q_3: ┤ H ├┤ Z ├┤ Tdg ├─┤ P(π/8) ├─┤1       ├────X──■───────
     └───┘└───┘└─────┘ └────────┘ └────────┘

Applying the circuit wrapper and transpiling to braket and cirq

In [13]:
wrapped_circuit = circuit_wrapper(qiskit_circuit)

In [14]:
braket_circuit = wrapped_circuit.transpile("braket")
print(f"{type(braket_circuit)}\n")
print(braket_circuit)

<class 'braket.circuits.circuit.Circuit'>

T  : |0|1|2 |     3     |4 |5|6|7| 8  |  9   |   10   |11|   12   |13|   14    |   15   |16|17|
                                                                                               
q0 : -H-X-S--Rx(0.79)----V--------SWAP--------C------------------------------------------------
                                  |           |                                                
q1 : -H-X-Si-Ry(1.57)----Vi-------|----SWAP---X------------------------------------------------
                                  |    |                                                       
q2 : -H-Y-T--Rz(2.36)----S--H-C-X-SWAP-|----U-Rx(1.57)-C--Rx(1.18)-X---------------------C--U--
                              | |      |               |           |                     |     
q3 : -H-Z-Ti-PHASE(0.39)-S----X-C-H----SWAP---U--------X--Ry(1.57)-C--Rx(-1.57)-Rz(1.57)-X--U--

T  : |0|1|2 |     3     |4 |5|6|7| 8  |  9   |   10   |11|   12   |13|   14    |   15   |16|

In [15]:
cirq_circuit = wrapped_circuit.transpile("cirq")
print(f"{type(cirq_circuit)}\n")
print(cirq_circuit)

<class 'cirq.circuits.circuit.Circuit'>

0: ───H───Z───T^-1───Z^(1/8)─────S────────────X───@───H───×───@────────
                                              │   │       │   │
1: ───H───Y───T──────Rz(0.75π)───S────────H───@───X───×───┼───@^0.25───
                                                      │   │
2: ───H───X───S^-1───Ry(0.5π)────X^-0.5───────────────┼───×───X────────
                                                      │       │
3: ───H───X───S──────Rx(0.25π)───X^0.5────────────────×───────@────────


Qubit indexing varies between packages, so some circuit diagrams appear flipped, but the matrix representations are equivalent.

To verify, we'll use the sdk's `circuits_allclose` function, which applies the qbraid `to_unitary` function mentioned above to each of two input circuits, and passes the matricies to `np.allclose`, and returns the result.

In [16]:
circuits_allclose(qiskit_circuit, braket_circuit) and circuits_allclose(
    braket_circuit, cirq_circuit
)

True

## Stress-testing against randomly generated circuits

As a final demo, we'll generate some even larger circuits, and do so randomly, to test the limits of the transpiler.

The qbraid-SDK has its own `random_circuit` function that takes in any supported package as an argument, but to show that there's no pre-processing or filtering going on behind the scenes, I'll use functions from cirq's testing module to generate circuits and to check equivalance after transpiling.

In [17]:
import cirq

kwargs = {
    "qubits": np.random.randint(8, 11),
    "n_moments": np.random.randint(8, 11),
    "op_density": np.random.randint(80, 100) / 100,
    "random_state": np.random.randint(1, 11),
}

circuit_start = cirq.testing.random_circuit(**kwargs)
start_u = circuit_start.unitary()
print("num qubits:", len(circuit_start.all_qubits()))
print("num moments:", len(circuit_start))
print("op density:", kwargs["op_density"])
print(f"matrix dim: {start_u.shape}\n")
print(circuit_start)

num qubits: 9
num moments: 10
op density: 0.98
matrix dim: (512, 512)

      ┌──────┐   ┌──┐   ┌───────────────┐   ┌──────┐   ┌───┐   ┌───┐   ┌──┐   ┌────┐   ┌───────┐   ┌──────┐
0: ─────────×──────×─────@────────────────────────×──────×────────×─────T──────@─────────@──────────S─────────
            │      │     │                        │      │        │            │         │
1: ────H────┼─────T┼─────@───────────────────H────┼─────×┼──────S─┼─────T──────┼────────X┼────────────────────
            │      │                              │     ││        │            │         │
2: ─────────┼──────×──────────iSwap──────────X────┼─────┼×───────@┼─────S──────┼X───────H┼──────────×─────────
            │                 │              │    │     │        ││            ││        │          │
3: ────Z────┼─────H──────Z────┼──────────────@────┼─────┼─@─────@┼┼─────S──────┼┼─@──────┼iSwap─────×─────────
            │                 │                   │     │ │     │││            ││ │      ││
4

Starting with this randomly generated circuit, we'll repeatedly apply the qbraid circuit wrapper and transpile from one supported package to the next until we arrive all the way back at a cirq circuit.

In [18]:
braket_circuit = circuit_wrapper(circuit_start).transpile("braket")
print(type(braket_circuit))
# print(f"\n{braket_circuit}")

<class 'braket.circuits.circuit.Circuit'>


In [19]:
pyquil_circuit = circuit_wrapper(braket_circuit).transpile("pyquil")
print(type(pyquil_circuit))
# print(f"\n{pyquil_circuit}")

<class 'pyquil.quil.Program'>


In [20]:
qiskit_circuit = circuit_wrapper(pyquil_circuit).transpile("qiskit")
print(type(qiskit_circuit))
# print(f"\n{qiskit_circuit}")

<class 'qiskit.circuit.quantumcircuit.QuantumCircuit'>


In [21]:
pennylane_circuit = circuit_wrapper(qiskit_circuit).transpile("pennylane")
print(type(pennylane_circuit))
# print(f"\n{pennylane_circuit}")

<class 'pennylane.tape.tape.QuantumTape'>


In [22]:
circuit_finish = circuit_wrapper(pennylane_circuit).transpile("cirq")
print(type(circuit_finish))
# print(f"\n{circuit_finish}")

<class 'cirq.circuits.circuit.Circuit'>


Computing the final unitary and checking its shape

In [23]:
finish_u = circuit_finish.unitary()
print(finish_u.shape)

(512, 512)


In [24]:
try:
    cirq.testing.assert_allclose_up_to_global_phase(start_u, finish_u, atol=1e-7)
    print("Test passed!")
except AssertionError:
    print("Test failed")

Test passed!


In [25]:
runtime = round(time() - start_notebook, 2)
print(f"Notebook ran in {runtime}s")

Notebook ran in 9.35s
